In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, split, lit, concat

In [2]:
def spark_session():
    spark = SparkSession.builder.master("local[*]") \
                        .appName("ny_bykeshares_analytics") \
                        .getOrCreate()
    return spark

In [3]:
def parse_date(df):
    df = df.withColumn('date', split(df['starttime'], ' ').getItem(0))
        
    df = df.withColumn('year', split(df['date'], '/').getItem(2))\
        .withColumn('day', split(df['date'], '/').getItem(1))\
        .withColumn('month', split(df['date'], '/').getItem(0)).drop('date')\
        .withColumn('date_for_join', concat(col('day'), lit('-'), ('month'), lit('-'), col('year'))).drop(col('month')).drop(col('day'))\
        .drop('year')
    
    return df

In [4]:
def handle_df_columns(df_1, df_2):
    flag = "NOK!"
    if  df_1.count()  == df_2.count():
        flag ="OK!"

    if flag != "OK!":
        raise Exception("Somethings is wrong with DFs")
    print(flag)

In [ ]:
from pyspark.sql.types import StructType, StructField, StringType,  FloatType, IntegerType

schema = StructType([
    StructField('tripduration', IntegerType()),
    StructField('start_station_name', StringType()),
    StructField('end_station_name', StringType()),
    StructField('user_type', StringType()),
    StructField('birthyear', IntegerType()),
    StructField('gender', IntegerType()),
    StructField('date', StringType()),
    StructField('maximum_temperature', FloatType()),
    StructField('minimum_temperature', FloatType()),
    StructField('average_temperature', FloatType()),
    StructField('precipitation', FloatType()),
    StructField('snow_fall', FloatType()),
    StructField('snow_depth', FloatType()),
])

In [15]:
def preanalytics_filter(df):
    df = df.filter(df['tripduration'] > 0)\
    .filter(df['tripduration'] < 5000)\
    .filter(df['maximum_temperature'] > -25).filter(df['maximum_temperature'] < 125)\
    .filter(df['minimum_temperature'] > -25).filter(df['minimum_temperature'] < 125)\
    .filter(df['average_temperature'] > -25).filter(df['average_temperature'] < 125)
    return df